In [1]:
import pandas as pd

df_a = pd.read_csv('data/TopArtists.csv')
df_t = pd.read_csv('data/TopTracks.csv')
df_g = pd.read_csv('data/TopGenres.csv')
df_m = pd.read_csv('data/MusicFeatures.csv')

In [2]:
df_a['user_id'].unique()

array([12153521253, 12120382831])

In [3]:
u1_a = df_a.loc[df_a['user_id'] == 12153521253]
u1_t = df_t.loc[df_t['user_id'] == 12153521253]
u1_g = df_g.loc[df_g['user_id'] == 12153521253]
u1_m = df_m.loc[df_m['user_id'] == 12153521253]

In [4]:
u2_a = df_a.loc[df_a['user_id'] == 12120382831]
u2_t = df_t.loc[df_t['user_id'] == 12120382831]
u2_g = df_g.loc[df_g['user_id'] == 12120382831]
u2_m = df_m.loc[df_m['user_id'] == 12120382831]

In [5]:
def top_to_dict(top_df):
    top_dict = {}
    top_dict['Short'] = top_df.loc[top_df['timeframe'] == 'Short'].to_dict('records')
    top_dict['Medium'] = top_df.loc[top_df['timeframe'] == 'Medium'].to_dict('records')
    top_dict['Long'] = top_df.loc[top_df['timeframe'] == 'Long'].to_dict('records')
    return top_dict

In [80]:
def get_artist_similarity(u1, u2, timeframe='Long'):
    df1 = u1.loc[u1['timeframe'] == timeframe, ['artist_id', 'artist', 'rank']]
    df2 = u2.loc[u2['timeframe'] == timeframe, ['artist_id', 'artist', 'rank']]
    df = df1.merge(df2, on=['artist_id', 'artist'], how='outer').fillna(0)
    df['base'] = calculate_score(df[df[['rank_x', 'rank_y']] > 0].min(axis=1))
    df.loc[(df['rank_x'] != 0) & (df['rank_y'] != 0), 'score'] = calculate_score(df[df[['rank_x', 'rank_y']] > 0].max(axis=1))
    df['score'] = df['score'].fillna(0)
    df = df.rename(columns={'rank_x': u1['user_id'].unique()[0], 'rank_y': u2['user_id'].unique()[0]})
    return df

def get_track_similarity(u1, u2, timeframe='Long'):
    df1 = u1.loc[u1['timeframe'] == timeframe, ['track_id', 'track', 'rank']]
    df2 = u2.loc[u2['timeframe'] == timeframe, ['track_id', 'track', 'rank']]
    df = df1.merge(df2, on=['track_id', 'track'], how='outer').fillna(0)
    df['base'] = calculate_score(df[df[['rank_x', 'rank_y']] > 0].min(axis=1))
    df.loc[(df['rank_x'] != 0) & (df['rank_y'] != 0), 'score'] = calculate_score(df[df[['rank_x', 'rank_y']] > 0].max(axis=1))
    df['score'] = df['score'].fillna(0)
    df = df.rename(columns={'rank_x': u1['user_id'].unique()[0], 'rank_y': u2['user_id'].unique()[0]})
    return df

def calculate_similarity(df):
    return round(df.sum()['score'] / df.sum()['base'], 4)

def calculate_score(rank, weight=16, shift=4):
    return weight / ((0.1 * rank + shift) ** 2) 

In [81]:
ma = get_artist_similarity(u1_a, u2_a)
ma.head()

,artist_id,artist,12153521253,12120382831,base,score
0,3OpWOvjJnk86geCHrAxkHA,Sungha Jung,1.0,0.0,0.951814,0.0
1,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,2.0,0.0,0.907029,0.0
2,04gDigrS5kc9YWfZHwBETP,Maroon 5,3.0,0.0,0.865333,0.0
3,44insiIQApkRaCMIbuaISJ,Joey Pecoraro,4.0,0.0,0.826446,0.0
4,7n2wHs1TKAczGzO7Dd2rGr,Shawn Mendes,5.0,0.0,0.790123,0.0


In [82]:
calculate_similarity(ma)

0.1414

In [83]:
mt = get_track_similarity(u1_t, u2_t)
mt.head()

,track_id,track,12153521253,12120382831,base,score
0,1RMJOxR6GRPsBHL8qeC2ux,Best Part (feat. H.E.R.),1.0,0.0,0.951814,0.0
1,4SkTfsjWUy2rp40d2Ayfmv,Tired Boy,2.0,0.0,0.907029,0.0
2,21cEse3PJxwXCRqsh2Xk47,Girlfriend,3.0,0.0,0.865333,0.0
3,70AvqwCfLnKMj3HvJDCXyW,Your Eyes,4.0,0.0,0.826446,0.0
4,5wyfaZpVP0lAuEWG9xJ3yA,Fly on the Wall,5.0,0.0,0.790123,0.0


In [84]:
calculate_similarity(mt)

0.0